In [1]:
import Pkg
Pkg.add("LightGraphs")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


# Doubly Connected Components

In [6]:
using LightGraphs

function merge_sets_union_find(set_of_sets::Set{Set{T}}) where T
    # Step 1: Map each element to a unique subset
    element_to_set = Dict{T, Set{T}}()
    for subset in set_of_sets
        for elem in subset
            element_to_set[elem] = subset
        end
    end

    # Step 2: Union subsets with overlapping elements
    for subset in set_of_sets
        # Find all subsets connected to the current subset
        merged_subset = Set(subset)
        for elem in subset
            merged_subset = union(merged_subset, element_to_set[elem])
        end

        # Update the mapping for all elements in the merged subset
        for elem in merged_subset
            element_to_set[elem] = merged_subset
        end
    end

    # Step 3: Collect unique subsets
    return Set(values(element_to_set))
end


merge_sets_union_find (generic function with 1 method)

In [7]:
function add_strays(graph::AbstractGraph, sets_of_nodes::Set{Set{Int}})
    # Collect all nodes in the given sets
    nodes_in_sets = reduce(union, sets_of_nodes, init=Set{Int}())
    
    # Get all nodes in the graph
    all_nodes = Set(1:nv(graph))
    
    # Find nodes not in any of the sets (strays)
    stray_nodes = setdiff(all_nodes, nodes_in_sets)
    
    # Add each stray node as its own set
    for stray in stray_nodes
        push!(sets_of_nodes, Set([stray]))
    end
    
    return sets_of_nodes
end

add_strays (generic function with 1 method)

In [8]:
function doubly_connected_components(graph)
    cycles = Set(Set(x) for x in filter(x->length(x)>2, simplecycles(DiGraph(graph))))
    doubly_connected_components_no_strays = merge_sets_union_find(cycles)
    return add_strays(graph, doubly_connected_components_no_strays)
end

doubly_connected_components (generic function with 1 method)

In [9]:
# Create a graph
graph = SimpleGraph(7)
add_edge!(graph, 1, 2)
add_edge!(graph, 2, 4)
add_edge!(graph, 4, 3)
add_edge!(graph, 3, 1)
add_edge!(graph, 2, 3)

add_edge!(graph, 4, 5)
add_edge!(graph, 5, 6)
add_edge!(graph, 6, 7)


doubly_connected_components(graph)


Set{Set{Int64}} with 4 elements:
  Set([5])
  Set([7])
  Set([4, 2, 3, 1])
  Set([6])

# Solver

In [32]:
using JuMP
using Gurobi
using LightGraphs  # For connectivity checks in the callback
using LinearAlgebra  # For distance calculations

# Example data (modify as needed)
n_facilities = 5  # Number of facilities
n_customers = 10  # Number of customers
k = 3  # Maximum number of facilities that can be selected

# Randomly generate coordinates for facilities and customers
facility_coords = rand(0:100, n_facilities, 2)  # Facility coordinates (x, y)
customer_coords = rand(0:100, n_customers, 2)  # Customer coordinates (x, y)

# Compute distances dynamically
function euclidean_distance(coord1, coord2)
    return norm(coord1 - coord2)
end

# Facility-to-customer distances
d = [euclidean_distance(facility_coords[i, :], customer_coords[j, :]) for i in 1:n_facilities, j in 1:n_customers]

# Facility-to-facility distances
D = [i == i_prime ? 0 : euclidean_distance(facility_coords[i, :], facility_coords[i_prime, :]) for i in 1:n_facilities, i_prime in 1:n_facilities]

# Cost parameters
C = rand(100:200, n_facilities)  # Facility setup costs
w = 5  # Cost per unit length of wire
F = 10  # Cost per unit length of fiber optic cable

# Initialize model
model = direct_model(Gurobi.Optimizer())

# Decision variables
@variable(model, x[1:n_facilities], Bin)  # Facility setup decision
@variable(model, 0 <= y[1:n_facilities, 1:n_customers] <= 1)  # Fractional connections
@variable(model, z[i=1:n_facilities, i_prime=i+1:n_facilities], Bin)  # Facility-to-facility connections only for i < i_prime

# Objective function: Minimize total costs
@objective(model, Min, sum(C[i] * x[i] for i in 1:n_facilities) +
                          sum(w * d[i, j] * y[i, j] for i in 1:n_facilities, j in 1:n_customers) +
                          sum(F * D[i, i_prime] * z[i, i_prime] for i in 1:n_facilities, i_prime in i+1:n_facilities))

# Constraints
# 1. Customer demand satisfaction
@constraint(model, [j=1:n_customers], sum(y[i, j] for i in 1:n_facilities) == 1)

# 2. Wires only for open facilities
@constraint(model, [i=1:n_facilities, j=1:n_customers], y[i, j] <= x[i])

# 3. Fiber optic cables only for open facilities
@constraint(model, [i=1:n_facilities, i_prime=i+1:n_facilities], z[i, i_prime] <= x[i])
@constraint(model, [i=1:n_facilities, i_prime=i+1:n_facilities], z[i, i_prime] <= x[i_prime])

# 4. Limit the number of selected facilities
@constraint(model, sum(x[i] for i in 1:n_facilities) <= k)
@constraint(model, sum(x[i] for i in 1:n_facilities) >= 3)

cb_calls = Cint[]
# 5. Connectivity callback to enforce at least 2 cables between partitions
function connectivity_callback(cb_data, cb_where::Cint)

    # You can reference variables outside the function as normal
    push!(cb_calls, cb_where)
    # You can select where the callback is run
    if cb_where != GRB_CB_MIPSOL
        return
    end
    

    println("WOOOOOOOOOOOOOO!!!")

    Gurobi.load_callback_variable_primal(cb_data, cb_where)

    println("HELLO 1!!!")

    # Get the values of the x and z variables
    x_vals = callback_value.(Ref(cb_data), x)
    z_vals = callback_value.(Ref(cb_data), z)
    z_vals = Dict((i, i_prime) => z_vals[i, i_prime] for i in 1:n_facilities for i_prime in i+1:n_facilities)

    println("HELLO 2!!!")

    # Identify selected facilities
    selected_facilities = [i for i in 1:n_facilities if x_vals[i] > 0.5]

    num_selected = length(selected_facilities)

    println("HELLO 3!!!")
    println("X: ", x_vals)
    println("Z: ", z_vals)
    println("SELECTED: ", selected_facilities)


    # Skip if fewer than 2 facilities are selected
    if num_selected <= 1
        return
    end

    # Translation dictionary for facility indices
    actual_to_simple = Dict(selected_facilities[i] => i for i in 1:num_selected)
    simple_to_actual = Dict(i => selected_facilities[i] for i in 1:num_selected)

    println("HELLO 4!!!")

    # Check for connectivity using a graph representation
    g = SimpleGraph(num_selected)
    for i in 1:num_selected
        for i_prime in i+1:num_selected
            if z_vals[(simple_to_actual[i], simple_to_actual[i_prime])] > 0.5
                add_edge!(g, i, i_prime)
            end
        end
    end

    println("HELLO 5!!!")

    # Find doubly connected components in the graph
    components = doubly_connected_components(g)
    println("COMPONENTS: ", components)

    # For each disconnected subset, enforce the connectivity constraint
    for component in components
        complement = setdiff(1:num_selected, component)
        if !isempty(component) && !isempty(complement)
            println("ADDING CONSTRAINT FOR COMPONENT: ", component)
            println([[min(simple_to_actual[i], simple_to_actual[i_prime]), max(simple_to_actual[i], simple_to_actual[i_prime])] for i in component, i_prime in complement])
            con = @build_constraint(sum(z[min(simple_to_actual[i], simple_to_actual[i_prime]), max(simple_to_actual[i], simple_to_actual[i_prime])] for i in component, i_prime in complement) >= 2)
            MOI.submit(model, MOI.LazyConstraint(cb_data), con)
        end
    end
end

# You _must_ set this parameter if using lazy constraints.
MOI.set(model, MOI.RawOptimizerAttribute("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), connectivity_callback)
optimize!(model)

# Output results
println("Optimal Objective Value: ", objective_value(model))
println("Facilities to open: ", [i for i in 1:n_facilities if value(x[i]) > 0.5])
println("Customer-facility connections (y):")
for i in 1:n_facilities, j in 1:n_customers
    if value(y[i, j]) > 1e-6  # Small threshold to filter non-zero values
        println("  Customer $j -> Facility $i: ", value(y[i, j]))
    end
end
println("Facility-facility connections (z):")
for i in 1:n_facilities, i_prime in i+1:n_facilities
    if value(z[i, i_prime]) > 0.5
        println("  Facility $i <-> Facility $i_prime")
    end
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 82 rows, 65 columns and 200 nonzeros
Model fingerprint: 0x13a3a3ea
Variable types: 50 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
WOOOOOOOOOOOOOO!!!
HELLO 1!!!
HELLO 2!!!
HELLO 3!!!
X: [1.0, 1.0, 1.0, 0.0, 0.0]
Z: Dict((2, 4) => 0.0, (1, 2) => 0.0, (4, 5) => 0.0, (2, 5) => 0.0, (1, 3) => 0.0, (3, 4) => 0.0, (1, 4) => 0.0, (2, 3) => 0.0, (1, 5) => 0.0, (3, 5) => 0.0)
SELECTED: [1, 2, 3]
HELLO 4!!!
HELLO 5!!!
COMPONENTS: Set(Set{Int64}[Set([3]), Set([1]), Set([2])])
ADDING CONSTRAINT FOR COMPONENT: Set([3])
[[